#Colab/Local Setup

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
%tensorflow_version 2.x
#local setup
#!pip install tensorflow-gpu==2.0.0 (or tensorflow==2.0.0)
#!pip install gdown

TensorFlow 2.x selected.


# Library imports and download data

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import gdown

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from sklearn.cluster import KMeans
import pickle

Note: Image Dataset (imgpath and condo_img_all.csv) not provided on Github

In [ ]:
gdown.download('https://drive.google.com/uc?export=download&id=1SPFSWBMjY0vBXX_0kL2XfoBwBvT0ZmmX', 'model.zip', quiet=False)

In [0]:
!unzip -qq model.zip -d .

In [0]:
basepath = Path('.')
imgpath = basepath/'data'/'image'
modelpath = basepath/'models'

# Load models for prediction

In [0]:
# load autoencoder
encoder = load_model(modelpath/'encoder_v2.h5', compile=False)

In [0]:
# load cluster model
kmeans_file = modelpath/'kmeans_v2.pkl'
kmeans_model = pickle.load(open(kmeans_file, 'rb'))

In [0]:
def predict_image_cluster(imagepath):
    """Predict cluster number of image

    Parameters:
    imagepath: image path using pathlib.Path format

    Returns:
    int: cluster number, ranges from 0 to 6

   """
    img = image.load_img(imagepath, target_size=(64, 64))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = img_data.flatten()
    img_data2 = []
    img_data2.append(img_data)
    img_data2 = np.array(img_data2)
    img_data2 = img_data2/255
    encoded = encoder.predict(img_data2)
    prediction = kmeans_model.predict(encoded)
    return int(prediction)

In [0]:
condo_img_all = pd.read_csv(basepath/'condo_img_all.csv', header=None)

In [0]:
condo_img_all.columns = ['image']

In [0]:
condo_img_all['cluster'] = condo_img_all['image']\
                            .apply(lambda x: 
                                   predict_image_cluster(imgpath/x))

In [0]:
condo_img_all.head()

# View predicted pictures

In [0]:
def plot_images(nrows, ncols, img_path_series, title):
    # settings
    nrows, ncols = 3, 3  # array of sub-plots
    figsize = [8, 8]     # figure size, inches

    # prep (x,y) for extra plotting on selected sub-plots
    xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
    ys = np.abs(np.sin(xs))           # absolute of sine

    # create figure (fig), and array of axes (ax)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    fig.suptitle(title)
    
    # plot simple raster image on each sub-plot
    for i, axi in enumerate(ax.flat):
        # i runs from 0 to (nrows*ncols-1)
        # axi is equivalent with ax[rowid][colid]
        tmp = cv2.imread(str(img_path_series[i]))
        axi.imshow(cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB))
        label = img_path_series[i].name
        label = label[:15] + (label[15:] and '..')
        axi.set_title(label)

    plt.tight_layout(True)
    plt.show()

In [0]:
for i in range(7):
    c = condo_img_all[condo_img_all['cluster'] == i].copy()
    c['image'] = c['image'].apply(lambda x: imgpath/x)

    c_paths = c['image'].unique()
    shuffled = c_paths[np.random.permutation(c_paths.shape[0])]
    plot_images(3, 3, shuffled, 'cluster ' + str(i))